In [3]:
import gensim
import os
import pickle

from sklearn.metrics.pairwise import cosine_similarity
import gensim
import numpy as np
from underthesea import word_tokenize
from gensim.models import LsiModel
from gensim.corpora import Dictionary

In [4]:
data_path = '/Users/admin/personal/master/ir/news_dataset/'
data_tokenized_path = '/Users/admin/personal/master/ir/news_dataset_tokenized/'

In [3]:
len(os.listdir(data_path))

43303

### tokenize (long task)

In [124]:
# corpus_text = []
# for file in os.listdir(data_path):
#     with open(data_path + file, 'r', encoding='utf-16') as f:
#         content = f.read()
#     content_tokenized = word_tokenize(content, 'text')
#     content_tokenized = content_tokenized.lower()
#     corpus_text.append(content_tokenized)
    
#     with open(data_tokenized_path + file, 'w', encoding='utf-16') as f:
#         f.write(content_tokenized)

### read tokenized docs

In [17]:
# corpus_text = []
# for file in os.listdir(data_tokenized_path):
#     try:
#         with open(data_tokenized_path + file, 'r', encoding='utf-16', errors="ignore") as f:
#             content = f.read()
#         corpus_text.append(content)
#     except:
#         pass

In [18]:
len(corpus_text)

43303

### build model LSI (long task)

In [6]:
# dictionary = Dictionary([doc.split() for doc in corpus_text])
# print(len(dictionary))


# corpus_bow = [dictionary.doc2bow(doc.split()) for doc in corpus_text]


# model = LsiModel(corpus_bow, id2word=dictionary, num_topics=500)

# vectorized_corpus = model[corpus_bow]
# vectorized_corpus = [np.array([v for i,v in doc]) for doc in vectorized_corpus]

242962


### saving

In [29]:
############# index - file name mapping
# index2name = {}
# name2index = {}

# for i, doc_name in enumerate(os.listdir(data_path)):
#     index2name[i] = doc_name
#     name2index[doc_name] = i

# with open('doc_name_index_mapping.pickle', 'wb') as handle:
#     pickle.dump((index2name, name2index), handle, protocol=pickle.HIGHEST_PROTOCOL)

In [33]:
"""LSI model including: 
* word dictionary (aka vocab)
* model (to vectorize query)
* vecorized_corpus (to lookup)
"""
# with open('lsi_model.pickle', 'wb') as handle:
#     pickle.dump((dictionary, vectorized_corpus, model), handle, protocol=pickle.HIGHEST_PROTOCOL)

## Serving

### query expansion

In [109]:
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('baomoi.model.bin', binary=True)

def expand_query(query):
    mean_vec = np.zeros(w2v_model.vector_size)
    for q in query.split():
        if q in w2v_model:
            mean_vec +=w2v_model[q]

    result = w2v_model.similar_by_vector(mean_vec, 5)
    result = [term for term, score in result if score > 0.6]
    return result

### query

In [4]:
with open('lsi_model.pickle', 'rb') as handle:
    dictionary, vectorized_corpus, model = pickle.load(handle)

In [5]:
with open('doc_name_index_mapping.pickle', 'rb') as handle:
    index2name, name2index = pickle.load(handle)

In [117]:
# obseravtion: những doc đá banh ko chứa từ "ronaldo" cũng dc trả về, do những doc có "ronaldo" đã được vectorize theo factors có pattern riêng cho bóng đá

query = 'cầu thủ ronaldo'

In [118]:
def query_corpus(query, use_expansion=False):
    expansion = []
    query_ = word_tokenize(query, 'text').lower()
    if use_expansion:
        expansion = expand_query(query_)
    
    query_bow = dictionary.doc2bow(query_.split() + expansion)
    query_lsi = np.array([v for i, v in model[query_bow]])
    
    result = list(enumerate(cosine_similarity([query_lsi], vectorized_corpus)[0]))
    result = sorted(result, key=lambda x:x[1], reverse=True)[:10]
    result = [(index2name[i], score) for i, score in result]
    return result

In [119]:
query_corpus(query)

[('TTO_060518_0658.txt', 0.5224573291445995),
 ('DTR_051220_1427.txt', 0.412814257261465),
 ('TTO_051227_0451.txt', 0.3827219307718617),
 ('TTO_050303_2148.txt', 0.3635003766961472),
 ('NLD_060606_1629.txt', 0.36096540058028753),
 ('TTO_060521_1725.txt', 0.35713785076727766),
 ('TTO_051205_1217.txt', 0.3383324732016544),
 ('NLD_060503_2359.txt', 0.33240354476686806),
 ('DTR_051220_1509.txt', 0.3309328525140842),
 ('DTR_051227_1504.txt', 0.3279902776319035)]

In [120]:
query_corpus(query, True)

[('TTO_060518_0658.txt', 0.5142212763034892),
 ('DTR_051220_1427.txt', 0.41958707307786813),
 ('TTO_051227_0451.txt', 0.37733228257730905),
 ('TTO_060521_1725.txt', 0.36499935754134005),
 ('TTO_050303_2148.txt', 0.3575417940627501),
 ('NLD_060606_1629.txt', 0.35367800057398646),
 ('DTR_051220_1509.txt', 0.3414000904997022),
 ('TTO_051205_1217.txt', 0.3333947368776999),
 ('DTR_051227_1504.txt', 0.3300274164401301),
 ('NLD_060503_2359.txt', 0.3287694289045251)]

In [6]:
def name2content(doc_name):
    content=''
    with open(data_path + doc_name, 'r', encoding='utf-16', errors="ignore") as f:
        content = f.read()
    return content

In [8]:
print(name2content('TTO_060518_0658.txt'))

Triệu tập thêm 3 cựu cầu thủ đội Cảng Sài Gòn

Ba cầu thủ này nằm trong danh sách 15 cầu thủ đã tham gia bán độ, tạo điều kiện cho đội SLNA giành chức vô địch mùa bóng 2000-2001. 
Trong vụ án này, cầu thủ Nguyễn Hữu Thắng của CLB SLNA đã giao cho cầu thủ Trương Tấn Hải của Cảng Sài Gòn 250 triệu đồng, sau đó Hải chia cho 15 cầu thủ tùy theo mức độ đóng góp trong phi vụ bán độ, thấp nhất là 4 triệu, cao nhất là 30 triệu. Cơ quan điều tra đã khởi tố năm cầu thủ của đội Cảng Sài Gòn có liên quan đến vụ án này.

M.Q.



In [121]:
for doc_name, s in query_corpus(query, True):
    print('===================')
    print(doc_name)
    with open(data_path + doc_name, 'r', encoding='utf-16', errors="ignore") as f:
        print(f.read())

TTO_060518_0658.txt
Triệu tập thêm 3 cựu cầu thủ đội Cảng Sài Gòn

Ba cầu thủ này nằm trong danh sách 15 cầu thủ đã tham gia bán độ, tạo điều kiện cho đội SLNA giành chức vô địch mùa bóng 2000-2001. 
Trong vụ án này, cầu thủ Nguyễn Hữu Thắng của CLB SLNA đã giao cho cầu thủ Trương Tấn Hải của Cảng Sài Gòn 250 triệu đồng, sau đó Hải chia cho 15 cầu thủ tùy theo mức độ đóng góp trong phi vụ bán độ, thấp nhất là 4 triệu, cao nhất là 30 triệu. Cơ quan điều tra đã khởi tố năm cầu thủ của đội Cảng Sài Gòn có liên quan đến vụ án này.

M.Q.

DTR_051220_1427.txt
Blatter phê phán Chelsea phung phí

Chủ tịch LĐBĐ thế giới (FIFA), ông Sepp Blatter đã phê phán mạnh mẽ việc Chelsea dùng quá nhiều tiền về đầu tư các cầu thủ nước ngoài. 
 
"Một CLB chỉ nên có 5 cầu thủ nước ngoài trong đội hình xuất phát, sau đó sẽ xây dựng một hệ thống đào tạo cầu thủ trẻ", Blatter nói trong bài phóng vấn trên tại chí chính thức của World Cup 2006. 
"Nếu Chelsea bị ép buộc chỉ có tối đa 5 cầu thủ nước ngoài trong đội